In [ ]:
import pandas as pd 
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.6f}'.format)


train_df = pd.read_csv('data/train.csv', index_col='Id')
test_df = pd.read_csv('data/test.csv', index_col='Id')

print(f"train data shape: {train_df.shape} \ntest data shape: {test_df.shape}")


print(f"Check null y: {train_df.SalePrice.isnull().sum()}")
print (f"Check NA y: {train_df.SalePrice.isna().sum()}")
#print duplicates
print(f"Check duplicates: {train_df.duplicated().sum()}")


train_df.head()


## 1a. drop feautres


In [600]:
train = train_df

test = test_df

def drop_feautres(df):
    df.drop(columns=["PoolArea", "MiscVal", '3SsnPorch', 'LowQualFinSF', 'BsmtFinSF2', 'EnclosedPorch'], inplace=True)

drop_feautres(train)


## 1b. Divide into X_train and Y_train

In [ ]:

# this is category not number 

def changeType(df, columnName):
    df[columnName] = df[columnName].astype(str)


changeType(train, 'MSSubClass')

# divide columns into numerical and categories


def set_column_types(df, set_type='train'):
    if set_type == 'train':
        num_cols = df.drop(columns='SalePrice').select_dtypes(include='number').columns
    else:
        num_cols = df.select_dtypes(include='number').columns

    cat_cols = df.select_dtypes(include='object').columns
    return num_cols, cat_cols

num_cols, cat_cols = set_column_types(train)


cat_cols

## 2a. Verify & handle Numerical Missing Values 


In [ ]:


def handle_missing_numerical(df):
    print(f"Missing numerical values:\n{df[num_cols].isna().sum().sort_values(ascending=False).head()}")

    # GarageYrBlt       81
    # MasVnrArea         8

    #### Lot Frontage
    df.loc[df.LotFrontage.isna(), ['Street', 'LotFrontage']]
    # LotFrontage      259 → for Na sreet is there so mean value will be applide
    df.LotFrontage.fillna(df.LotFrontage.mean(), inplace=True)
    #### Garage 
    # Garage type Nan means no garage so year should 0
    df.loc[df.GarageYrBlt.isna(), df.columns.str.contains('Garage', case=False)]
    df.GarageYrBlt.fillna(0, inplace=True)
    # Masonry
    # masnory type is none so area should be 0
    df.loc[df.MasVnrArea.isna(), df.columns.str.contains('MasVnr', case=False)]
    df.MasVnrArea.fillna(0, inplace=True)
    print(f"\nAfter processing:\n{df[num_cols].isna().sum().sort_values(ascending=False).head()}")

handle_missing_numerical(train)



## 2b. handle outliers 


In [ ]:

# Verify features aftecting price 
corr = pd.DataFrame({'SalePrice': train_df.drop(columns=cat_cols).corr()['SalePrice'].sort_values(ascending=False)}).iloc[1:]
corr.style.background_gradient(cmap='YlOrRd')



In [604]:
# Based on graph for similicty for outlier i'm picking top features: 
cols_to_verify = ['GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'LotArea' ]



In [ ]:
plt.figure(figsize=(25, 80))

for count, feature in enumerate(cols_to_verify):
    plt.subplot(12, 3, count+1)
    plt.scatter(train[cols_to_verify].loc[:, feature], train.SalePrice)
    plt.xlabel(feature, fontsize=16)
    plt.ylabel('Sale Price', fontsize=14)
plt.show()





In [ ]:
def identify_outliers(df):
    """
    Identify outliers based on specific conditions for each feature
    Returns the indices of outlier rows
    """
    outlier_indices = []
    
   # GrLivArea outliers (large area, low price)
    condition1 = (df['GrLivArea'] > 4000) & (df.SalePrice < 300000)
    indices1 = df[condition1].index.tolist()
    outlier_indices.extend(indices1)
    print(f"Found {len(indices1)} outliers with large GrLivArea but low price")
    
    # GrLivArea high-end outliers (expensive houses)
    condition2 = (df['GrLivArea'] > 4000) & (df.SalePrice > 650000)
    indices2 = df[condition2].index.tolist()
    outlier_indices.extend(indices2)
    print(f"Found {len(indices2)} outliers with large GrLivArea and very high price")
    
    # TotalBsmtSF outliers (large basement, low price)
    condition3 = (df['TotalBsmtSF'] > 3000) & (df.SalePrice < 250000)
    indices3 = df[condition3].index.tolist()
    outlier_indices.extend(indices3)
    print(f"Found {len(indices3)} outliers with large TotalBsmtSF but low price")
    
    # 1stFlrSF outliers (large first floor, unusual price)
    condition4 = (df['1stFlrSF'] > 3000)   & (df.SalePrice < 300000)
    indices4 = df[condition4].index.tolist()
    outlier_indices.extend(indices4)
    print(f"Found {len(indices4)} outliers with unusually large 1stFlrSF")
    
    # GarageArea outliers (large garage, low price)w
    condition5 = (df['GarageArea'] > 1200) & (df.SalePrice < 200000)
    indices5 = df[condition5].index.tolist()
    outlier_indices.extend(indices5)
    print(f"Found {len(indices5)} outliers with large GarageArea but low price")
    
    # GarageCars outliers (4 car garages)
    condition6 = (df['GarageCars'] == 4)
    indices6 = df[condition6].index.tolist()
    outlier_indices.extend(indices6)
    print(f"Found {len(indices6)} outliers with 4 car garages")
    
    # Very high-priced houses (general luxury outliers)
    condition7 = (df.SalePrice > 650000)
    indices7 = df[condition7].index.tolist()
    outlier_indices.extend(indices7)
    print(f"Found {len(indices7)} outliers with very high prices (>$650,000)")

    # Lot Area
    indices9 = df[df.LotArea > 100000].index.tolist()
    outlier_indices.extend(indices9)
    print(f"Found {len(indices9)} outliers with very large lot area > 100k")


    # Remove duplicates (a row might satisfy multiple conditions)
    outlier_indices = list(set(outlier_indices))
    print(f"Total unique outliers found: {len(outlier_indices)}")
    
    return outlier_indices




outlier_indices = identify_outliers(train)


train = train.drop(outlier_indices)
# have to remove both to mach shape 
print(f"Original shape: {train_df.shape}, Clean shape: {train.shape}")

print(f"\nAfter processing:\n{train[num_cols].isna().sum().sort_values(ascending=False).head()}")



## 3. Verify Categorical Columns


In [ ]:

def fill_missing_categoricals(df):
    print(df[cat_cols].isna().sum().sort_values(ascending=False).head(5))


    # just fill with 'Na' 
    df[cat_cols] = df[cat_cols].fillna('Na')
    print(f"\n\nAfter processing: \n{df[cat_cols].isna().sum().sort_values(ascending=False).head(5)}")
    
fill_missing_categoricals(train)


In [ ]:
ordinal_features_cols = ['Alley', 'LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
        'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'PavedDrive', 'KitchenQual']

ordinal_mappings = {
        # Alley access
        'Alley': {np.nan: 0 , 'NA': 0, 'Grvl': 1, 'Pave': 2},
        
        # Lot shape
        'LotShape': {'IR3': 0, 'IR2': 1, 'IR1': 2, 'Reg': 3},
        
        # Utilities
        'Utilities': {'NoSeWa': 0, 'NoSewr': 1, 'AllPub': 3},
        
        # Land Slope
        'LandSlope': {'Sev': 0, 'Mod': 1, 'Gtl': 2},
        
        # Quality ratings (consistent pattern across several features)
        'ExterQual': {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
        'ExterCond': {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
        'BsmtQual': {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
        'BsmtCond': {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
        'HeatingQC': {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
        'FireplaceQu': {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
        'GarageQual': {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
        'GarageCond': {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
        'PoolQC': {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
        
        # Basement exposure
        'BsmtExposure': {'NA': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4},
        
        # Basement finished types
        'BsmtFinType1': {'NA': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
        'BsmtFinType2': {'NA': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
        
        # Home functionality
        'Functional': {'Sal': 1, 'Sev': 2, 'Maj2': 3, 'Maj1': 4, 'Mod': 5, 'Min2': 6, 'Min1': 7, 'Typ': 8},
        
        # Garage finish
        'GarageFinish': {'NA': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3},
        
        # Fence quality
        'Fence': {'NA': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4},

        #Kitchen quality
        'KitchenQual' : {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd' : 4, 'Ex' : 5},

        # paved driveway 
        'PavedDrive' : {'N': 1, 'P': 2, 'Y': 3}
    }

ordinal_features = []
binary_features = []
multi_features = []


def categorise_features(train):
    if len(ordinal_features) > 0 and len(binary_features) > 0 and len(multi_features) > 0:
        return
    
    """
    Categorise features into 3 groups for further processing
    """
    
    cat_info = {}


    for col in cat_cols:    
        unique_vals =  train[col].value_counts(dropna=False).shape[0]
        missing_vals = train[col].isna().sum()
        sample_vals = train[col].value_counts().head(3).index.tolist()
        
        cat_info[col] = {
            'unique_values': unique_vals,
            'missing_values': missing_vals,
            'examples': sample_vals
        }

    for col, info in cat_info.items():
        if col in ordinal_features_cols:
            ordinal_features.append(col)
        elif info['unique_values'] == 2:
            binary_features.append(col)
        else:
            multi_features.append(col)
     

def process_binary_features(train):
        """
        Modify binary descriptive features to be 0 or 1 
        """
        binary_mappings = {
            'Street': {'Pave': 1, 'Grvl': 0},
            'CentralAir': {'Y': 1, 'N': 0}
        }
        for feature, mapping in binary_mappings.items():
            train[feature] = train[feature].map(mapping)
    
    # Ensure any unexpected values (if they appear) are handled
        if train[feature].isna().any():
            # print(f"Warning: Unexpected values in {feature} - filling with mode")
            train[feature] = train[feature].fillna(train[feature].mode()[0])

        # for feature in binary_features:
        #     print(f"{feature} after encoding: {train[feature].value_counts().to_dict()}") 

def process_ordinal_features(train):
    """
    Modify ordinal descriptive features to numerical values
    """
    # for each entry in ordinal_features, apply the mapping
    for feature in ordinal_features:
        # print(f"DEBUG: current feature: {feature}")
        train[feature] = train[feature].map(ordinal_mappings[feature])
        # print(f"{feature} after encoding: {train[feature].value_counts().to_dict()}")
        # Ensure any unexpected values (if they appear) are handled
        if train[feature].isna().any():
            # print(f"Warning: Unexpected values in {feature} - filling with mode")
            train[feature] = train[feature].fillna(train[feature].mode()[0])
        # print(f"{feature} after encoding: {train[feature].value_counts().to_dict()}")

def process_multi_features(train):
    encoded_features = []
    for feature in multi_features:
        multi = pd.get_dummies(train[feature], prefix=f'{feature[0:4] + feature[-1]}', prefix_sep='_')
        encoded_features.append(multi)
    encoded_df = pd.concat(encoded_features, axis=1)
    train = train.drop(columns=multi_features)
    train = pd.concat([train, encoded_df], axis=1)
    return train
     


def process_all_cateorical_features(df):
    categorise_features(df)
    print (f"\nBinary features: {binary_features}\n")
    print (f"Ordinal features: {ordinal_features}\n")
    print (f"Multi features: {multi_features}\n")
    process_binary_features(df)
    process_ordinal_features(df)
    df = process_multi_features(df)
    return df


train = process_all_cateorical_features(train)

train.head()



## 4. Transform output 
- only y_train

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# transform the price : 
# present it : 
def plot_dist(column, title):
    plt.figure(figsize=(10, 6))
    sns.histplot(column, kde=True)
    plt.title(title)
    plt.xlabel("Price")
    plt.ylabel('Frequency')
    plt.show()
plot_dist(train.SalePrice, "Before log mod: Distribution of Sale Price") 
train.SalePrice = np.log(train.SalePrice)
plot_dist(train.SalePrice ,"After log mod: Distribution of Sale Price")

train.SalePrice.to_csv('data/y_train.csv', index=False)



## 5. Scale Features


### 5.B Apply Scaling only on  numerical_continues_cols

In [610]:
from sklearn.preprocessing import StandardScaler



def scale_selected_numerical_features(df) -> pd.DataFrame:
    """Drops columns which are not continues and scales rest numerical

    Args:
        df (_type_): _description_
    """

    if 'SalePrice' in df.columns:
        df.drop(columns=['SalePrice'], inplace=True)

    
    numerical_discrete_cols = ['OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath' , 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 
                 'Fireplaces', 'GarageCars']
    
    numerical_df = df[num_cols]
    numerical_continues_cols = numerical_df.drop(columns=numerical_discrete_cols).columns

    # create separate df for scaling 
    numerical_continues_df = df[numerical_continues_cols]

    # drop orginal columns → those will be scaled and re-added 
    df.drop(columns=numerical_continues_cols, inplace=True)
    # print(f"No of columns after drop: {len(df.columns)}")

    # process scaling 
    scaler = StandardScaler()
    numerical_continues_scalled_array = scaler.fit_transform(numerical_continues_df)
    numerical_continues_scalled_df = pd.DataFrame(numerical_continues_scalled_array, columns=numerical_continues_df.columns, index=numerical_continues_df.index)

    # now need to apply log transformaiton for few features 
    # Features with very high scaling values that need transformation
    log_transform_features = [
        'LotFrontage',  'LotArea', 
        'MasVnrArea', 'ScreenPorch', 'OpenPorchSF'
    ]  

    # print(f"Is any null before log transform: {numerical_continues_scalled_df.isna().sum().sort_values(ascending=False)}")


#####  DEBUG
    for feature in ['LotFrontage', 'LotArea']:
        # Print min and max values
        print(f"{feature} min: {numerical_continues_scalled_df[feature].min()}")
        print(f"{feature} max: {numerical_continues_scalled_df[feature].max()}")
        
        # Check if any values are close to -1
        near_negative_one = sum((numerical_continues_scalled_df[feature] < -0.99) & 
                                (numerical_continues_scalled_df[feature] > -1.01))
        print(f"{feature} values near -1: {near_negative_one}")
        
        # Check for infinite values
        inf_values = sum(np.isinf(numerical_continues_scalled_df[feature]))
        print(f"{feature} infinite values: {inf_values}")
        
        # Look at a few examples of values that might be causing problems
        problematic = numerical_continues_scalled_df[numerical_continues_scalled_df[feature] < -0.9]
        if not problematic.empty:
            print(f"Sample problematic values for {feature}:")
            print(problematic[feature].head())



#########

    for feature in log_transform_features:
        numerical_continues_scalled_df[f'{feature}_log'] = np.log1p(numerical_continues_scalled_df[feature])
        numerical_continues_scalled_df.drop(columns=[feature], inplace=True)

    # print(f"Mean values of scaled df\n\n{numerical_continues_scalled_df.mean().sort_values(ascending=False)}")
    # print(f"MAX values of scaled df\n\n{numerical_continues_scalled_df.max().sort_values(ascending=False)}")
    


    ##  very unclean workaround → fill na calused by log transform with 0 
    
    numerical_continues_scalled_df.fillna(0, inplace=True)

    return numerical_continues_scalled_df



# print(f"No of columns = {len(train.columns)}")


In [ ]:

# Train df 

new_train_partital_df = scale_selected_numerical_features(train)
# print(f"Is any null in new partial log transform: {new_train_partital_df.isna().sum().sum()}")



# print(f"Partial columns lenght={len(new_train_partital_df.columns)}")

X_train = pd.merge(train, new_train_partital_df, left_index=True, right_index=True)

# # save the data to csv
# X_train = X_train.reset_index(names="Id")
# X_train.isna().sum().sort_values(ascending=False)
# print(f"No of columns = {len(X_train.columns)}")


# TEST DF

In [ ]:
# TEST DF 
test = pd.read_csv('data/test.csv', index_col='Id')

#1 changeType
changeType(test, 'MSSubClass')

#2 drop_feautres
drop_feautres(test)

#3 set_column_types
num_cols, cat_cols = set_column_types(test, "test")

#4 handle_missing_numerical(test)
handle_missing_numerical(test)

#5 fill_missing_categoricals
fill_missing_categoricals(test)

test = process_all_cateorical_features(test)



new_test_partital_df = scale_selected_numerical_features(test)
print(f"Partial columns lenght={len(new_train_partital_df.columns)}")

X_test = pd.merge(test, new_test_partital_df, left_index=True, right_index=True)

X_test.head()
# print(f"No of columns = {len(X_train.columns)}")



### 5c merge dfs 

In [ ]:
# FIND DIFFS IN COLUMNS BETWEEN X_TRAIN and X_TEST
set(X_train.columns.to_list()).symmetric_difference(set(X_test.columns.to_list()))

In [614]:
exted_columns = X_test.filter(like='Exted').columns.to_list()
extet_replacements = []

for col in exted_columns:
    # Create corresponding 'Extet' name from 'Exted'
    new_name = col.replace('Exted', 'Extet')
    extet_replacements.append(new_name)

rename_dict = dict(zip(exted_columns, extet_replacements))
X_test.rename(columns=rename_dict, inplace=True)



# Example: Fix Electrical column inconsistencies
elecl_columns = X_test.filter(like='Elecl').columns.to_list()
if elecl_columns:
    # Map to corresponding columns in train dataset
    elecl_replacements = [col.replace('Elecl', 'Electrical') for col in elecl_columns]
    rename_dict = dict(zip(elecl_columns, elecl_replacements))
    X_test.rename(columns=rename_dict, inplace=True)


X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)
X_test.fillna(0, inplace=True)

X_train.to_csv('data/X_train_cleaned.csv', index=True, index_label='Id')
X_test.to_csv('data/X_test_cleaned.csv', index=True, index_label='Id')


In [ ]:
set(X_train.columns.to_list()).symmetric_difference(set(X_test.columns.to_list()))

In [ ]:
X_test.isna().sum().sort_values(ascending=False)